# Введение в работу с Azure Machine Learning (AML)

---

## Введение
[Azure Machine Learning](https://docs.microsoft.com/en-us/azure/machine-learning/overview-what-is-azure-ml) предоставляет возможности по работе с данными, версионированию датасетов, моделей и логированию результатов обучения. Весь этот функционал доступен из Python SDK. В данном ноутбуке мы рассмотрим:
* подключение к AML Workspace
* доступ к датасету
* создание эксперимента и отслеживание метрик
* выбор лучшей модели

Для работы вам потребуется [**Azure ML SDK**](https://docs.microsoft.com/en-us/python/api/overview/azure/ml/?view=azure-ml-py). AML SDK предустановлен в следующих ***conda*** окружениях:
* с префиксом `azureml_py36_*` - Python 3.6
* `py37_default`- Python 3.7

***Окружение `azureml_py36_automl` является рекомендованным.***

Для каждой команды мы заранее создали Azure ML Workspace и зарегистрировали в нем датасет `train_ds`, содержащий исходные данные.

---

## Подключение к AML Workspace

[`Workspace`](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) — основной объект для хранения и доступа к экспериментам, датасетам и другим артифактам вашей работы.
Импортируем SDK и подключимся к командному Workspace. 

*Ячейка ниже не требует редактирования. После авторизации возвращает номер вашей команды и объект Workspace.*

In [1]:
import azureml.core
from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.core import Experiment, Workspace

# Check core SDK version number
print("You are currently using version", azureml.core.VERSION, "of the Azure ML SDK")
print("")

# Log In to Azure ML Workspace
interactive_auth = InteractiveLoginAuthentication(tenant_id="76f90eb1-fb9a-4446-9875-4d323d6455ad")

ws = Workspace.from_config(auth=interactive_auth)
print('Workspace name: ' + ws.name, sep='\n')

You are currently using version 1.5.0 of the Azure ML SDK

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code CKCULJKMK to authenticate.
Interactive authentication successfully completed.
Workspace name: team20


---

## Доступ к датасету

Для доступа к датасету `train_ds` используем метод [`get_by_name`](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.dataset.dataset?view=azure-ml-py#get-by-name-workspace--name--version--latest--). Он вернет объект класса [`TabularDataset`](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.data.tabulardataset?view=azure-ml-py#methods). Который далее можно материализовать в Pandas Dataframe.

**!NB** Помните о том, что вы используете shared вычислительные ресурсы на команду. Импортировать весь датасет целиком каждому участнику может быть не оптимально.

Для предобработки и анализа части датасета можно использовать следующие методы:
* [take(count)](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.data.tabulardataset?view=azure-ml-py#take-count-)
* [take_sample(probability, seed=None)](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.data.tabulardataset?view=azure-ml-py#take-sample-probability--seed-none-)
* [skip(count)](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.data.tabulardataset?view=azure-ml-py#skip-count-)

In [2]:
from azureml.core import Dataset

# get the dataset from Azure ML Workspace
aml_dataset = Dataset.get_by_name(ws, 'train_ds', version='latest')
pdf = aml_dataset.take(100).to_pandas_dataframe()

**Сохранить локально**

Если в ходе работы вам потребуется сохранить датасет локально на виртуальной машине, можно использовать методы [`to_csv_files()`](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.data.tabulardataset?view=azure-ml-py#to-csv-files-separator-----) и [`download()`](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.data.filedataset?view=azure-ml-py#download-target-path-none--overwrite-false-).

In [ ]:
aml_dataset.to_csv_files(separator=',').download(target_path='/home/user_X/data/')

**Регистрация новой версии датасета**

По мере работы с данными внешний вид датасета будет трансформироваться. Для сохранения и регистрации новой версии датасета внутри AML Workspace необходимо:
* загрузить файл с новым датасетом в Workspace 
* зарегистрировать новую версию датасета

In [ ]:
# To create a dataset from an in memory pandas dataframe,
# write the data to a local file, like a csv, and create your dataset from that file.
local_path = '~/data/new_data.csv'
pdf.to_csv(local_path)

In [ ]:
# upload the local file to a datastore on the cloud
from azureml.core import Dataset

# get the datastore to upload prepared data
datastore = ws.get_default_datastore()

# upload the local file from src_dir to the target_path in cloud datastore
datastore.upload(src_dir='/home/user_X/data', target_path='data',overwrite=True)
# create a dataset referencing the cloud location
dataset = Dataset.Tabular.from_delimited_files(datastore.path('data/new_data.csv'))
# register a new dataset in Azure ML Workspace
registerered_ds = dataset.register(workspace=ws,
                                 name='new_data',
                                 description='Team_XX transformed dataset',
                                 tags={'CSV':'True'})

---
## Обучение модели и логирование метрик

В ходе обучения будем использовать AML для сохранения полезной информации о полученных результатах внутри Experiment. Обычно эксперимент заключает в себе наработки по проверке одиночной гипотезы. Experiment содержит набор итераций обучения - [`Runs`](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.run(class)?view=azure-ml-py).  Для логирования метрик и моделей:
* Cоздаем [`Experiment`](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.experiment(class)?view=azure-ml-py) внутри нашего Workspace
* Используем [`start_logging`](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.experiment(class)?view=azure-ml-py#start-logging--args----kwargs-) для создания нового Run внутри Experiment
* Для логирования необходимых параметров и метрик используем [`run.log()`](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.run(class)?view=azure-ml-py#log-name--value--description----). В дальнейшем мы сможем просмотреть все метрики через [портал](https://ml.azure.com/) Azure ML.
* Сохраняем полученную модель в директории **outputs**, которая автоматически загружается в ваш Workspace при завершении Run.
* Используем [`run.complete()`](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.run(class)?view=azure-ml-py#complete--set-status-true-) чтобы указать что текущая итерация обучения (Run) завершена и все результаты могут быть отправлены в Workspace.

**!NB**  Обратите внимание на доступные методы ***run.log_\**** Они позволяют добавлять в Run различные виды метрик (например, изображения, таблицы). Также приветствуется использование тэгов.

In [8]:
from sklearn.datasets import load_diabetes
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import joblib

X, y = load_diabetes(return_X_y = True)
columns = ['age', 'gender', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
data = {
    "train":{"X": X_train, "y": y_train},        
    "test":{"X": X_test, "y": y_test}
}

print ("Data contains", len(data['train']['X']), "training samples and",len(data['test']['X']), "test samples")

Data contains 353 training samples and 89 test samples


In [9]:
# Get an experiment object from Azure Machine Learning
experiment = Experiment(workspace=ws, name="my-first-experiment_mark")

# Create a run object in the experiment
run =  experiment.start_logging()
# Log the algorithm parameter alpha to the run
run.log('alpha', 0.03)

# Create, fit, and test the scikit-learn Ridge regression model
regression_model = Ridge(alpha=0.03)
regression_model.fit(data['train']['X'], data['train']['y'])
preds = regression_model.predict(data['test']['X'])

# Output the Mean Squared Error to the notebook and to the run
print('Mean Squared Error is', mean_squared_error(data['test']['y'], preds))
run.log('mse', mean_squared_error(data['test']['y'], preds))

# Save the model to the outputs directory for capture
model_file_name = 'outputs/model.pkl'

joblib.dump(value = regression_model, filename = model_file_name)

# upload the model file explicitly into artifacts 
run.upload_file(name = model_file_name, path_or_stream = model_file_name)

# Complete the run
run.complete()

Mean Squared Error is 3424.900315896017


### Просмотр результатов Run
Azure Machine Learning хранит все сведения о Run в облаке Microsoft Azure. Получить доступ к текущему Run можно прямо из кода или из [портала](https://ml.azure.com/) Azure ML.

In [10]:
run

Experiment,Id,Type,Status,Details Page,Docs Page
my-first-experiment_mark,bb7b970f-5e9c-459e-a765-0f392c79f789,,Running,Link to Azure Machine Learning studio,Link to Documentation


### Пример: подбор параметров
Пример ниже демонстрирует реализацию механизма подбора параметра **alpha** в ходе обучения. Для каждого значения Alpha создается свой Run. По завершении эксперимента это позволит нам выбрать лучшую модель.

**!NB** `with experiment.start_logging() as run` автоматически вызовет `run.complete()` в конце каждого цикла.

In [ ]:
import numpy as np
from tqdm import tqdm

# list of numbers from 0 to 1.0 with a 0.05 interval
alphas = np.arange(0.0, 1.0, 0.05)

# try a bunch of alpha values in a Linear Regression (Ridge) model
for alpha in tqdm(alphas):
    # create a bunch of runs, each train a model with a different alpha value
    with experiment.start_logging() as run:
        # Use Ridge algorithm to build a regression model
        regression_model = Ridge(alpha=alpha)
        regression_model.fit(X=data["train"]["X"], y=data["train"]["y"])
        preds = regression_model.predict(X=data["test"]["X"])
        mse = mean_squared_error(y_true=data["test"]["y"], y_pred=preds)

        # log alpha, mean_squared_error and feature names in run history
        run.log(name="alpha", value=alpha)
        run.log(name="mse", value=mse)

        # Save the model to the outputs directory for capture
        joblib.dump(value=regression_model, filename='outputs/model.pkl')


### Просмотр результатов Experiment
Аналогично с результатами Run, мы можем получить обзор о проведенном эксперименте. [Портал](https://ml.azure.com/)  Azure ML предоставляет возможность строить графики по залогированным метрикам. Эти графики можно использовать в презентации своей работы.

In [15]:
# now let's take a look at the experiment in Azure portal.
experiment

Name,Workspace,Report Page,Docs Page
my-first-experiment_mark,team20,Link to Azure Machine Learning studio,Link to Documentation


### Выбор лучшей модели

Для выбора наиболее успешной модели сравним полученный набор Runs внутри Experiment. Для каждого Run получим его ID используя свойство `id` , и проанализируем метрики вызвав `run.get_metrics()`.  

Так как каждый Run внутри эксперимента может иметь различный набор метрик, нужно удостовериться что он имеет нужную нам -  **mse**.  Для поиска лучшего Run создадим dictionary ID - MSE.

Затем используем метод [`tag`](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.run(class)?view=azure-ml-py#tag-key--value-none-) и присвоим лучшему Run значение "Best Run". Данный тэг поможет нам на этапе регистрации модели.

In [ ]:
runs = {}
run_metrics = {}

# Create dictionaries containing the runs and the metrics for all runs containing the 'mse' metric
for r in tqdm(experiment.get_runs()):
    metrics = r.get_metrics()
    if 'mse' in metrics.keys():
        runs[r.id] = r
        run_metrics[r.id] = metrics

# Find the run with the best (lowest) mean squared error and display the id and metrics
best_run_id = min(run_metrics, key = lambda k: run_metrics[k]['mse'])
best_run = runs[best_run_id]
print('Best run is:', best_run_id)
print('Metrics:', run_metrics[best_run_id])

# Tag the best run for identification later
best_run.tag("Best Run")

---
## Реестр моделей
После того как мы завершили обучение и выбрали Run содержащий модель с лучшей метрикой, зарегистрируем полученную модель в Azure ML Workspace. Класс [`Model`](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.model(class)?view=azure-ml-py) позволяет работать с реестром моделей внутри Azure ML. При помощи него можно автоматизировать версионирование и деплоймент, создать единый репозитарий моделей внутри компании.

### Регистрация
На предыдущем шаге мы выбрали Run содержащий лучшую модель. Каждый run имеет ассоциированную с ним структуру файлов собранных во время исполнения. Выберем файл представляющий нашу модель. Используем [`run.get_file_names()`](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.run(class)?view=azure-ml-py#get-file-names--) для получения списка всех файлов связанных с Run, и затем метод [`run.register_model()`](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.run(class)?view=azure-ml-py#register-model-model-name--model-path-none--tags-none--properties-none--model-framework-none--model-framework-version-none--description-none--datasets-none--sample-input-dataset-none--sample-output-dataset-none--resource-configuration-none----kwargs-) для регистрации модели внутри реестра моделей Azure ML.

При использовании `run.register_model()` необходимо указать `model_name` отражающий особенности нашего эксперимента и `model_path` к модели внутри Run. В данном случае путь будет получен при помощи `run.get_file_names()`.

In [ ]:
# View the files in the run
for f in best_run.get_file_names():
    print(f)
    
# Register the model with the workspace
model = best_run.register_model(model_name='best_model', model_path='outputs/model.pkl')

После регистрации модель становится доступна в реестре моделей Azure ML. При регистрации моделей с одинаковым именем Azure ML автоматически создаст новую версию существующей модели, обеспечив возможность версионирования и отката.  Метод `Model.list()` отобразит все зарегистрированные модели внутри Workspace. Они могут быть отфильтрованы по имени, тэгам, или другим атрибутам модели.

In [ ]:
# Find all models called "best_model" and display their version numbers
from azureml.core.model import Model
models = Model.list(ws, name='best_model')
for m in models:
    print(m.name, m.version)

## Полезные ссылки

* [Azure Machine Learning Python SDK](https://docs.microsoft.com/en-us/python/api/overview/azure/ml/?view=azure-ml-py)
* [Tools included on this VM](https://docs.microsoft.com/en-us/azure/machine-learning/data-science-virtual-machine/tools-included)